In [9]:
from typing import Iterator
from pathlib import Path

DATASET = Path("../data/masked.jsonl")

MODEL = "ufal/robeczech-base"

In [10]:
import pandas as pd

df = pd.read_json(DATASET)
df

,sentence,error,masked
0,jako sboristka Chloe,jako sboristka Chloe,jako sboristka Chloe
1,Z obavy před odkloněním části pracovních sil n...,Z obavy před odkloněním částIi pracovních sil ...,Z obavy před odkloněním [MASK] pracovních sil ...
2,Naopak na jižní polokouli je viditelná pouze d...,Naopak na jižní polokouli je viditelná pouze d...,Naopak na jižní polokouli je viditelná pouze d...
3,JeanCharles se také zajímal o vědu a mechaniku,JeanCharlesj seI také zajímal o qědu a mechaniku,[MASK] [MASK] také zajímal o [MASK] a mechaniku
4,Jakmile je dosaženo požadované velikosti elekt...,Jakmile je dosaženo požadované velikosti elekt...,Jakmile je dosaženo požadované velikosti elekt...
...,...,...,...
9995,Spor proto řešila Kapitulní konzistoř v Praze,Spor proto ře4šila Kapitulní konzitoř v aze,Spor proto [MASK] Kapitulní konzitoř v [MASK]
9996,19661971 Smrt Walta a Roye Disneyových a otevř...,19661971 Srt WaltaC a Roye Disneyových a otevř...,19661971 [MASK] [MASK] a Roye Disneyových a [M...
9997,Přestože byla požádána o účast v dalších dílec...,Přestože byla požádána o účast v dalších dílec...,Přestože byla požádána o účast v dalších dílec...
9998,Následně nebyl zvolen ani za řadového místopře...,Následně nebyl zvolen ani za řadového místopře...,Následně nebyl zvolen ani za řadového místopře...


In [11]:
from transformers import pipeline

corrector = pipeline("fill-mask", model=MODEL, tokenizer=MODEL, device=0, top_k=1)

In [12]:
def create_masks(row: pd.Series) -> list[str]:
    masked = row["masked"].split(" ")
    error = row["error"].split(" ")

    result: list[str] = []

    for i, w in enumerate(masked):
        _error = error.copy()
        if w == "[MASK]":
            try:
                _error[i] = "[MASK]"
                result.append(" ".join(_error))
            except IndexError:
                return []

    return result

### Example usage

In [13]:
masks = create_masks(df.iloc[6])

print(masks)

[]


In [14]:
corrector("Baal Baal Lev je krátký hudební televizní film z roku 1997 [MASK] Eytna FoxeA podle scénáře který napsal Gal OhovskOhovski")

[{'score': 0.4931256175041199,
  'token': 33,
  'token_str': ' od',
  'sequence': 'Baal Baal Lev je krátký hudební televizní film z roku 1997 od Eytna FoxeA podle scénáře který napsal Gal OhovskOhovski'}]

### Define function for furter processing

In [15]:
from tqdm.auto import tqdm
from transformers import Pipeline
from dataclasses import dataclass
from util import optimal_split, masked_correct


@dataclass
class PredictionData:
    sentence: str
    error: str
    masked: str
    predictions: list[str]


class ProcessPrediction:
    def __init__(self, pipeline: Pipeline):
        self.pipeline = pipeline

    def process_prediction(self, data: list[PredictionData]) -> Iterator[str]:
        raise NotImplementedError("Implement this method")

def join_masked(row: list[str]) -> str | None:
    is_all_floats = all(isinstance(item, float) for item in row)
    if is_all_floats:
        return None

    masks: list[list[str]] = [
        masked.split(" ") for masked in row
    ]

    sentence: list[str] = []

    for elements in zip(*masks):
        if "[MASK]" in elements:
            sentence.append("[MASK]")
        else:
            sentence.append(elements[0])
    
    return " ".join(sentence)

def create_dataset(pred: ProcessPrediction) -> pd.DataFrame:
    global df

    final_dataset = pd.DataFrame()
    
    dfs = optimal_split(df)

    for _df in tqdm(dfs, desc="Filling masks for shards", total=len(dfs)):
        _df["masked"] = _df.apply(create_masks, axis=1)

        # explode dataset so we can use it in the pipeline
        _df = _df.explode("masked")
        _df["replace"] = None
        _df.reset_index(drop=True, inplace=True)

        not_nan = _df[_df["masked"].notnull()].index

        df_to_process = _df.loc[not_nan]
        df_to_process.reset_index(drop=True, inplace=True)

        # apply pipeline
        predictions = corrector(df_to_process["masked"].to_list(), batch_size=32)

        # process predictions
        prediction_data: list[PredictionData] = [
            PredictionData(
                sentence=row.sentence,
                error=row.error,
                masked=row.masked,
                predictions=[p["token_str"].strip() for p in pred]
            ) for row, pred in zip(df_to_process.itertuples(), predictions)
        ]

        for j, prediction in enumerate(pred.process_prediction(prediction_data)):
            df_to_process.loc[j, "replace"] = prediction

        _df.loc[not_nan] = df_to_process
        
        # implode and merge
        _df = _df.groupby(["sentence", "error"]).agg(
            {
                "masked": list,
                "replace": list,
            }
        ).reset_index()

        # process masked sentences
        _df["masked"] = _df["masked"].apply(join_masked)

        final_dataset = pd.concat([final_dataset, _df])

    return masked_correct(final_dataset)

### Experiment 1
We will replace `[MASK]` with the suggestion that has biggest score.

In [16]:
class Experiment1(ProcessPrediction):
    def process_prediction(self, data: list[PredictionData]) -> Iterator[str]:
        for row in data:
            yield row.predictions[0]

corrector = pipeline("fill-mask", model=MODEL, tokenizer=MODEL, device=0, top_k=1)
proc = Experiment1(corrector)

result = create_dataset(proc)
result.to_json(DATASET.parent / "experiment-1.jsonl", orient="records", lines=True, index=False)

/home/adam/projects/KNN/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Filling masks for shards:   0%|          | 0/10 [00:00<?, ?it/s]

### Experiment 2

In [ ]:
from nltk import edit_distance

class Experiment2(ProcessPrediction):
    def process_prediction(self, data: list[PredictionData]) -> Iterator[str]:
        for row in data:
            error = row.error.split(" ")
            masked = row.masked
            predictions = row.predictions
            index = masked.split(" ").index("[MASK]")

            invalid_word = error[index]

            best = float('inf')
            _prediction = ""
            for prediction in predictions:
                if edit_distance(prediction, invalid_word) < best:
                    best = edit_distance(prediction, invalid_word)
                    _prediction = prediction

            yield _prediction


corrector = pipeline("fill-mask", model=MODEL, tokenizer=MODEL, device=0, top_k=50)
proc = Experiment2(corrector)
result = create_dataset(proc)
result.to_json(DATASET.parent / "experiment-2.jsonl", orient="records", lines=True, index=False)

/home/adam/projects/KNN/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Filling masks for shards: 100%|██████████| 10/10 [01:21<00:00,  8.12s/it]
